<a href="https://colab.research.google.com/github/ChaesongYun/YT_Title_Predictor/blob/main/Youtube_Title_Predictor_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 Fine Tuning

클리닝이 끝난 데이터를 BERT 자연어처리 인공지능 모델을 이용해서 fine tuning 작업을 해준다.<br>
구글에서 pre-tranined된 BERT 모델을 받아준 뒤 데이터를 맞춤 제작을 한다.
<br>
맞춤제작이 끝나면 그 모델 유튜브 제목 퀄리티를 예측할 수 있다!(●'◡'●)

1. Pre-tranined tokenizer 다운 받기
2. Data sampling. 데이터 섞기
3. 데이터셋을 training용과 validation용으로 나누기
4. 인풋 데이터 만들기
5. 학습모델 모델링
6. 모델 compile
7. 모델 fit
8. 모델 평가하기
9. 모델 다운로드

In [ ]:
# 필요한 라이브러리 가져오기
!pip install transformers # Hugging Face에서 가져온다
import transformers
from transformers import BertTokenizer

# 선행된 Tokenizer 다운받기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

import tensorflow as tf
import numpy as np
import pandas as pd